## SAT BBR 
* Issue [#220](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/220)
* This [notebook](220_SAT_BBR.ipynb)

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-10-04 19:06:04


In [5]:
from string import Template
from datetime import datetime as dt
import html as html_mod

def add_about_box(
    m,
    issue_number: int,
    map_name: str,
    created_date: str | None = None,
    repo: str = "salgo60/Stockholm_Archipelago_Trail",
    collapsed: bool = False,
    offset_px=(10, 54),  # (top, left) – 54px för att hamna snällt bredvid zoom
):
    """Superenkel, robust About-box som alltid syns."""
    if created_date is None:
        created_date = dt.now().strftime("%Y-%m-%d %H:%M")

    map_dom_id = m.get_name()
    box_id     = f"sat-about-{map_dom_id}"
    header_id  = f"{box_id}-hdr"
    issue_url  = f"https://github.com/{repo}/issues/{issue_number}"
    top, left  = offset_px
    collapsed_class = "sat-about-collapsed" if collapsed else ""

    links = [
        ("SAT Dashboard", "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_dashboard_latest.html"),
        ("Project repo issues", "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue"),
        ("Trail on OSM (rel 19012437)", "https://www.openstreetmap.org/relation/19012437"),
        ("Trail on Wikicommons", "https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail"),
        ("Official page", "https://stockholmarchipelagotrail.com/"),
        ("Unofficial FB group", "https://www.facebook.com/groups/2875020699552247"),
        ("Visit Sweden", "https://traveltrade.visitsweden.com/plan/news-sweden/Stockholm-Archipelago-Trail/"),
    ]
    links_html = "".join(
        f'<div><a href="{html_mod.escape(u)}" target="_blank" style="text-decoration:none;">🔗 {html_mod.escape(t)}</a></div>'
        for t, u in links
    )

    tpl = Template(r"""
<style>
  .sat-about {
    position: fixed; z-index: 10000;
    top: ${top}px; left: ${left}px;
    background: rgba(255,255,255,0.97);
    border: 2px solid #666; border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.25);
    font: 12px/1.35 -apple-system, system-ui, Segoe UI, Roboto, Helvetica, Arial, sans-serif;
    min-width: 240px; max-width: 320px; pointer-events: auto;
  }
  .sat-about-header { cursor: pointer; padding: 8px 10px; font-weight: 700;
    display: flex; align-items: center; gap: 6px; user-select: none;
    background: rgba(248,248,248,.9); border-bottom: 1px solid #e5e7eb; }
  .sat-about-body { padding: 8px 10px 10px 10px; }
  .sat-about-collapsed .sat-about-body { display: none; }
  .sat-chevron { margin-left: auto; transition: transform .15s ease-in-out; }
  .sat-about-collapsed .sat-chevron { transform: rotate(-90deg); }
  .sat-links { margin-top: 6px; padding-top: 6px; border-top: 1px solid #e5e7eb; }
</style>

<div id="${box_id}" class="sat-about ${collapsed_class}">
  <div id="${header_id}" class="sat-about-header" title="Click to collapse/expand">
    <span>ℹ️ About</span><span class="sat-chevron">▸</span>
  </div>
  <div class="sat-about-body">
    <div style="font-weight:700;margin-bottom:4px;">Stockholm Archipelago Trail Map</div>
    <div>Issue: <a href="${issue_url}" target="_blank">#${issue_number}</a>&nbsp;&nbsp; Map: ${map_name}</div>
    <div>Created: ${created_date}</div>
    <div>Latest updates: saved as <i>_latest.html</i></div>
    <div class="sat-links">${links_html}</div>
  </div>
</div>

<script>
(function(){
  var boxId = "${box_id}";
  var hdrId = "${header_id}";
  var storageKey = "satAboutCollapsed_${map_dom_id}_#${issue_number}";

  function setCollapsed(box, collapsed) {
    if (!box) return;
    if (collapsed) box.classList.add("sat-about-collapsed");
    else box.classList.remove("sat-about-collapsed");
    try { localStorage.setItem(storageKey, collapsed ? "1" : "0"); } catch(e) {}
  }

  function init(){
    var box = document.getElementById(boxId);
    var hdr = document.getElementById(hdrId);
    if (!box || !hdr) return;

    try {
      var stored = localStorage.getItem(storageKey);
      if (stored === "1") setCollapsed(box, true);
      if (stored === "0") setCollapsed(box, false);
    } catch(e) {}

    hdr.addEventListener("click", function(e){
      e.stopPropagation();
      setCollapsed(box, !box.classList.contains("sat-about-collapsed"));
    });
  }

  if (document.readyState === "loading") {
    document.addEventListener("DOMContentLoaded", init);
  } else {
    init();
  }
})();
</script>
""")

    html_code = tpl.substitute(
        box_id=box_id,
        header_id=header_id,
        issue_number=issue_number,
        issue_url=issue_url,
        map_name=html_mod.escape(map_name),
        created_date=created_date,
        links_html=links_html,
        collapsed_class=collapsed_class,
        map_dom_id=map_dom_id,
        top=top, left=left,
    )
    m.get_root().html.add_child(folium.Element(html_code))


In [6]:
import folium
import geopandas as gpd
import fiona
import datetime, os
from shapely.geometry import shape
from SPARQLWrapper import SPARQLWrapper, JSON

OUTPUT_PREFIX = "SAT_BBR"

# --- helper: hämta Wikidata Q-id och label via P1260 ---
def get_wikidata_item(bbr_id):
    """Slår upp ett RAÄ/BBR-ID i Wikidata P1260"""
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent="SAT-map/0.1")
    sparql.setReturnFormat(JSON)
    sparql.setQuery(f"""
    SELECT ?item ?itemLabel WHERE {{
      ?item wdt:P1260 "raa/bbr/{bbr_id}" .
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "sv,en". }}
    }}
    """)
    try:
        results = sparql.query().convert()
        if results["results"]["bindings"]:
            b = results["results"]["bindings"][0]
            return b["item"]["value"].split("/")[-1], b["itemLabel"]["value"]
    except Exception:
        pass
    return None, None


# --- data paths ---
sat_path = "SAT_full.geojson"
raa_path = "kulturhistoriskt_inventerad_bebyggelse_stockholm.gpkg"

# Läs in SAT-leden
sat = gpd.read_file(sat_path).to_crs("EPSG:3006")
sat_buffer = sat.buffer(1000).union_all() # buffer 1000 m

# Läs RAÄ-data via fiona
matches = []
with fiona.open(raa_path, layer="kulturhistoriskt_inventerad_bebyggelse_stockholm_polygon") as src:
    for feat in src:
        geom = shape(feat["geometry"])
        if geom.intersects(sat_buffer):
            matches.append(feat)

raa_gdf = gpd.GeoDataFrame.from_features(matches, crs="EPSG:3006").to_crs("EPSG:4326")
sat = sat.to_crs("EPSG:4326")

# --- karta ---
m = folium.Map(location=[59.4, 18.5], zoom_start=9, tiles="OpenStreetMap")

# SAT-leden blå linje
folium.GeoJson(
    sat,
    name="SAT-leden",
    style_function=lambda x: {"color": "blue", "weight": 3}
).add_to(m)

# RAÄ-objekt
for _, row in raa_gdf.iterrows():
    props = row.drop("geometry").to_dict()
    namn = props.get("namn", "Okänt namn")
    alt_namn = props.get("alternativnamn")
    uuid = props.get("id")
    bbr = props.get("bebyggelsenummer")
    
    # länkar
    raa_link = f"https://app.raa.se/open/bebyggelse/bebyggelseobjekt/{uuid}"
    osm_ref = f"ref:raa.se:bebyggelseobjekt={uuid}"
    
    wd_q, wd_label = (None, None)
    if bbr:
        wd_q, wd_label = get_wikidata_item(bbr)
    
    # popup HTML
    html = f"<h4>{namn}</h4>"
    if alt_namn:
        html += f"<i>{alt_namn}</i><br>"
    html += f'<a href="{raa_link}" target="_blank">RAÄ Bebyggelseregister</a><br>'
    html += f"<i>OSM-taggförslag:</i> <code>{osm_ref}</code><br>"
    if wd_q:
        html += f'<a href="https://www.wikidata.org/wiki/{wd_q}" target="_blank">Wikidata: {wd_q} – {wd_label}</a><br>'
    
    # metadata-lista
    meta_fields = [
        ("Objekttyp", props.get("objekttyp_namn")),
        ("Skyddstyp", props.get("skyddstyp_namn")),
        ("Beslutsdatum", props.get("beslutsdatum")),
        ("Klassificering", props.get("klassificering")),
        ("Årtal", f"{props.get('nybyggnad_tillkomstar_from')}–{props.get('nybyggnad_tillkomstar_tom')}"),
        ("Ändamål", props.get("andamal_huvudgrupp")),
        ("Kommun", props.get("kommunnamn")),
        ("Län", props.get("lansnamn")),
        ("Stomme", props.get("stomme_material")),
        ("Fasad", props.get("fasad_material")),
        ("Tak", f"{props.get('taktackning_material')} / {props.get('takform')}"),
    ]
    html += "<hr><b>Metadata:</b><ul style='font-size:13px;'>"
    for label, val in meta_fields:
        if val and str(val) != "None":
            html += f"<li><b>{label}:</b> {val}</li>"
    html += "</ul>"
    
    folium.Marker(
        [row.geometry.centroid.y, row.geometry.centroid.x],
        tooltip=namn,
        icon=folium.Icon(color="green", icon="info-sign", prefix="glyphicon", icon_size=(36,36)),
        popup=folium.Popup(html, max_width=400)
    ).add_to(m)

# About-box
add_about_box(m, issue_number=120, map_name="SAT BBR")

# --- spara ---
os.makedirs("output", exist_ok=True)
ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
out_ts = os.path.join("output", f"{OUTPUT_PREFIX}_{ts}.html")
out_latest = os.path.join("output", f"{OUTPUT_PREFIX}_latest.html")

m.save(out_ts)
m.save(out_latest)

print("Klar:")
print("  ", out_ts)
print("  ", out_latest)

m


Klar:
   output/SAT_BBR_20251004_191430.html
   output/SAT_BBR_latest.html


In [7]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


AttributeError: module 'datetime' has no attribute 'now'